# Introduction to Impresso's embedding essentials

<a target="_blank" href="https://colab.research.google.com/github/impresso/impresso-datalab-notebooks/blob/main/workshop_resources/ws4-embeddings/explore_introduction-to-embeddings.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

If something doesn't work, you can [report a problem](https://github.com/impresso/impresso-datalab-notebooks/blob/main/reporting-problems.md).

## What is this notebook about?

This notebook is designed as a **practical introduction to the Impresso embedding ecosystem**. 

The **first section** introduces the idea of text embeddings and demonstrates how textual content can be transformed into vector representations. This serves as a conceptual foundation for the rest of the material.

The **second section** guides users through two fundamental questions: how embeddings can be inspected to better understand their structure, and how similarity functions work in practice. These two exercises help make the abstract notion of vectors tangible, before moving toward more advanced retrieval workflows.

The **third section** focuses on Impresso’s embedding search capabilities. It explains how to perform text retrieval across multiple languages, how to limit searches by language, and how images can also be retrieved via their own embeddings. Users are then shown how to access both text and image embeddings directly from the Impresso platform, as well as how to retrieve the associated media objects. 

## What you will learn?

 - Embed texts and images;
 - Retrieve embeddings directly from the Impresso database;
 - Search related texts and images within Impresso.

## Useful resources

- [Impresso Python Library](https://impresso.github.io/impresso-py/)
- [Impresso Huggind Face](https://ipyleaflet.readthedocs.io/en/latest/index.html)


## Prerequisites

Run the following cells to install the required package and to connect to Imrpesso API:

> If you are working with Google Colab, you may need to restart the kernel. Go to *Runtime* and select *Restart session*. 

In [1]:
# Impresso Python package with embeddings search feature

!pip install impresso


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [2]:
# Connecting to Impresso API

from impresso import connect, OR, AND, DateRange
impresso_session = connect('https://dev.impresso-project.ch/public-api/v1')


Click on the following link to access the login page: https://dev.impresso-project.ch/datalab/token
 - 🔤 Enter your email/password on this page.
 - 🔑 Once logged in, a secret token will be generated for you.
 - 📋 Copy this token and paste it into the input field below. Then press "Enter". 👇🏼.

🎉 You are now connected to the Impresso API!  🎉
🔗 Using API: https://dev.impresso-project.ch/public-api/v1


## 1. Embedding a text

Embedding a text means using a **neural network** to turn it into a **high-dimensional vector** that allows efficient comparison across languages and beyond surface wording. Let's start by generating such an embedding right away!

In [3]:
embedding = impresso_session.tools.embed_text(text="Schumann the politician", target="text")
embedding

'gte-768:9wisvV3v1D0Pb1M7CWzzPEv727stV8u9ttGePBKJNTsdoCU9QyzcvOIYBzyG+ss88Xp6vRjOkz2amm69+5JTPU7zqD3hMxE9STrHPTg+tLzQhjQ9qJERPLuVPjx7Pca7iL12vSlZxbwfw889WK2RvWtL5r10C3g9CC6RvdGPGb3SubM8UXh+PSXwQj1urAe9FVKKPUN/vTxXwK69+uNQvJ3PF72wyxG9xaInvTa7m7x9DT48HJI5PMEldby2QkU8CDQnOcQCaz3+C2q9l/pbPT5pgryrU4E8bPvyPOcu8jw2pOe9jEMxPceb0D227dy88vZFvLGcHbsPhxS9uSnSvDm0FzwqkBO7m/03PSxh87yh7Jo9/OmRPVN3gLzH1AU9s3v7O+LA/Tz/0ns8pFHkPcxVSbyoj6885vCdPbtzwjxDLki9e4UYPC5YuDxEbsA8hN0gvAb6bzz7ccc5/SaIvXCUQbyLVLk8B1d1Pa3zEbxsEw88XI4nvcZ0WLsWOzY8O+ynvEfaLDuoGlo8vm3IvFyRJT2+xSM95p4yPc9obLzOSeW8qTYQvTWWyTv4/Cs72nvIuLcaJDwGn4W9U+bCvCavVb3y9f+85HxdvSJrVz05YV49vKcSvdVQSj3hKmk8BLiyu9j7Ar3VCK68s9/6vBcQxb0O73K8E2uQvbW1kr0HjSo92YAIvc6iLz3QkHo88arZPF8YD72qXTM8X88hvZWNEL3Mtx89Bv0tOxvxgDlHlwm9IaqGPFnttr1Xxry8X52ZPA22uDxrGqG82G51PPNtJz0hmiE7/7llPLZCND0aJvk8xeH/vL2SCbuyC/e8MLIivQNcQD0GC8k7BbSkvFGLdT3dMVI7STLGPBkCqrwJgkC9ud2SOyZ1TLqtgBc94iRaPdbK4LyVZii9spk5PfwuBD38Yr27C6nWvOgmFTkA8uK8uk0IPekZJ70n3d28fdkZvSz5fb34gY69fnSRvU0bTjwkH4M

### Does it look like a vector?

Having inspected the generated embedding, one might wonder what these weird characters and numbers mean: ```gte-768:9wisvV3v1D0Pb1M7CWzzPEv727stV8u9ttGePBKJNTsd...```

The reason for why this embedding does not look like a vector of numbers is rather simple: **It's encoded in a data-efficient format**.
Let's introduce two helper functions to be able to map back-and-forth between vectors and data-friendly encodings.

In [4]:
# Convert our embedding string to a vector with 768 dimensions:

import base64
import struct

def string2vector(embedding_string):
    # convert base64 string to a float array
    _, arr = embedding_string.split(':')
    arr = base64.b64decode(arr)
    embedding_vector = [struct.unpack('f', arr[i:i+4])[0] for i in range(0, len(arr), 4)]
    return embedding_vector

def vector2string(vec, prefix="gte-768"):
    # pack floats into bytes
    arr = b''.join(struct.pack('f', x) for x in vec)
    # encode bytes to base64 string
    encoded = base64.b64encode(arr).decode('utf-8')
    # return in same format as original ("prefix:encoded_string")
    return f"{prefix}:{encoded}"

string2vector(embedding)[:5]

[-0.08400147408246994,
 0.1039721742272377,
 0.003226224100217223,
 0.02971460111439228,
 -0.006713306065648794]

In [5]:
# Boolean expression to verify conversion correctness:

embedding == vector2string(string2vector(embedding))

True

### What is Similarity Function?

To understand **why embeddings are powerful for comparing and searching texts**, we’ll define a simple similarity function and use it to compare three examples.

In [7]:
def vector_similarity(vector1, vector2):
    return sum([x*y for x, y in zip(vector1, vector2)])

text1 = "Schumann the politician"
text2 = "Schumann der politiker"
text3 = "Schumann the composer"

# text1 vs text2
s1 = vector_similarity(string2vector(impresso_session.tools.embed_text(text=text1, target="text")),
                       string2vector(impresso_session.tools.embed_text(text=text2, target="text")))
# text1 vs text3
s2 = vector_similarity(string2vector(impresso_session.tools.embed_text(text=text1, target="text")),
                       string2vector(impresso_session.tools.embed_text(text=text3, target="text")))

s1, s2

(0.9331190426943782, 0.8527833427887213)

> Notice that, even though the first two texts are written in the same language and share more tokens, they are less similar in the semantic vector space than a text in another language that expresses the same idea. In other words, **embeddings capture meaning rather than simple word overlap**.
This type of **similarity function** is precisely what Impresso uses internally when performing embedding-based text search. In the next part of the notebook, we will build on this insight and experiment with embedding-based retrieval.

## 2. Impresso's embedding search

The idea of search in a nutshell: First, we embed our text, and then the similarity magic finds us the most similar documents in a dataset. Thanks to embeddings, **search can be performed across languages and through all documents in Impresso!**

### Text Retrieval Across Languages 🇫🇷🇩🇪🇬🇧🇱🇺

Remember our text embeddings are cross-lingual: this means we can search across languages!
You can embed any text that you like.
So feel **free to overwrite the string in the `text` variable**.

In [9]:
text = "Schumann the politician"
embedding = impresso_session.tools.embed_text(text=text, target="text")
impresso_session.search.find(
  embedding=embedding,
  limit=5
)

,copyrightStatus,type,sourceMedium,title,topics,transcriptLength,totalPages,languageCode,isOnFrontPage,publicationDate,issueUid,countryCode,providerCode,mediaUid,mediaType,hasOLR,ocrQualityScore,pageNumbers,collectionUids,entities.locations,entities.persons,entities.organisations,entities.newsAgencies,mentions.locations,mentions.persons,mentions.organisations,mentions.newsAgencies
uid,,,,,,,,,,,,,,,,,,,,,,,,,,,
EXP-1963-09-06-a-i0005,in_cpy,ar,print,Nombreux témoignages de sympathie à la famille...,"[{'uid': 'tm-fr-all-v2.0_tp21_fr', 'relevance'...",313,1,fr,True,1963-09-06T00:00:00+00:00,EXP-1963-09-06-a,CH,SNL,EXP,newspaper,True,1.00,[1],[],"[{'uid': '2-54-Metz', 'count': 2}, {'uid': '2-...","[{'uid': '2-50-Robert_Schuman', 'count': 9}, {...","[{'uid': '2-53-Marché_commun', 'count': 2}]",[],"[{'surfaceForm': 'Metz', 'mentionConfidence': ...","[{'surfaceForm': 'M. Robert Schuman', 'mention...","[{'surfaceForm': 'Marché commun', 'mentionConf...",[]
luxland-1963-09-06-a-i0046,in_cpy,ar,print,Mon ami Robert Schuman,"[{'uid': 'tm-fr-all-v2.0_tp99_fr', 'relevance'...",364,1,fr,False,1963-09-06T00:00:00+00:00,luxland-1963-09-06-a,LU,BNL,luxland,newspaper,True,1.00,[2],[],"[{'uid': '2-54-Luxembourg', 'count': 1}, {'uid...","[{'uid': '2-50-Robert_Schuman', 'count': 3}]",[],[],"[{'surfaceForm': 'Luxembourg', 'mentionConfide...","[{'surfaceForm': 'ami Robert Schuman', 'mentio...",[],[]
GDL-1953-10-23-a-i0001,in_cpy,ar,print,le de M. ScliuiMii,"[{'uid': 'tm-fr-all-v2.0_tp87_fr', 'relevance'...",678,1,fr,True,1953-10-23T00:00:00+00:00,GDL-1953-10-23-a,CH,SNL,GDL,newspaper,True,0.99,[1],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Setting Language Restrictions 🇫🇷

Yes, our embeddings can retrieve all kinds of data across languages!
But this is not always what we want.
Sometimes, we want to **find texts only from a target language of our choice**.
Here's how that can be done in Impresso: Set `country` to either `FR` `DE` `EN` or `LB` (Luxembourgish):

In [11]:
impresso_session.search.find(
  country='FR',
  embedding=embedding,
  limit=5
)

,copyrightStatus,type,sourceMedium,title,topics,transcriptLength,totalPages,languageCode,isOnFrontPage,publicationDate,issueUid,countryCode,providerCode,mediaUid,mediaType,hasOLR,ocrQualityScore,pageNumbers,collectionUids,entities.locations,entities.persons,entities.organisations,entities.newsAgencies,mentions.locations,mentions.persons,mentions.organisations,mentions.newsAgencies
uid,,,,,,,,,,,,,,,,,,,,,,,,,,,
excelsior-1920-06-19-a-i0024,in_cpy,ar,print,LA CRISE ALLEMANDE : M. FEHRENBACH N'OBTIENT P...,"[{'uid': 'tm-fr-all-v2.0_tp98_fr', 'relevance'...",462,1,fr,False,1920-06-19T00:00:00+00:00,excelsior-1920-06-19-a,FR,BNF,excelsior,newspaper,True,1.0,[3],[],"[{'uid': '2-54-Versailles', 'count': 1}]",[],"[{'uid': '2-53-Berlin', 'count': 1}]",[],"[{'surfaceForm': 'centre BERLIN', 'mentionConf...","[{'surfaceForm': 'ancien maire de Strasbourg, ...","[{'surfaceForm': 'en-Brisgau', 'mentionConfide...",[]
oeuvre-1938-02-25-a-i0003,in_cpy,ar,print,Le discours du Chancelier,"[{'uid': 'tm-fr-all-v2.0_tp29_fr', 'relevance'...",364,1,fr,True,1938-02-25T00:00:00+00:00,oeuvre-1938-02-25-a,FR,BNF,oeuvre,newspaper,True,1.0,[1],[],"[{'uid': '2-54-Front', 'count': 1}, {'uid': '2...","[{'uid': '2-50-Kurt_Schuschnigg', 'count': 2}]",[],[],"[{'surfaceForm': 'Front', 'mentionConfidence':...","[{'surfaceForm': 'M. Schuschnigg, chancelier f...",[{'surfaceForm': 'parti national-socialiste au...,[]
oeuvre-1930-11-17-a-i0047,in_cpy,ar,print,Le chef populiste bavarois fait appel à l'éner...,"[{'uid': 'tm-fr-all-v2.0_tp98_fr', 'relevance'...",144,1,fr,False,1930-11-17T00:00:00+00:00,oeuvre-1930-11-17-a,FR,BNF,oeuvre,newspaper,True,1.0,[3],[],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Image retrieval with embeddings

Suppose we have an image or a piece of text and we want to find out whether there are any images within Impresso that are similar to it. In that case, we can rely on the **Impresso image search to identify and retrieve visually or semantically related images**.

#### Use-case A: We want to find similar images to our text

This is also called a "multi-modal" search, since input and retrieved data are from two different modalities (image, text).
Hence, we set the `target="multimodal"`:

In [13]:
impresso_session.images.find(
  embedding=impresso_session.tools.embed_text(text="Berlin", target="multimodal"),
  limit=3
)

,issueUid,previewImage,date,contentItemUid,pageNumbers,mediaSourceRef.uid,mediaSourceRef.name,mediaSourceRef.type,imageTypes.visualContent,previewUrl,imageTypes.visualContentType
uid,,,,,,,,,,,
JDG-1954-02-13-a-i0214,JDG-1954-02-13-a,,1954-02-13,JDG-1954-02-13-a-i0211,[16],JDG,Journal de Genève,newspaper,Image,"https://dev.impresso-project.ch/media/iiif/JDG-1954-02-13-a-p0016/2094,3529,682,570/max/0/default.jpg",Map - Geopolitical
IMP-1958-12-17-a-i0009,IMP-1958-12-17-a,,1958-12-17,IMP-1958-12-17-a-i0002,[1],IMP,L'Impartial,newspaper,Image,"https://dev.impresso-project.ch/media/iiif/IMP-1958-12-17-a-p0001/448,621,554,484/max/0/default.jpg",Map - Geopolitical
EXP-2009-10-17-a-i0258,EXP-2009-10-17-a,,2009-10-17,EXP-2009-10-17-a-i0254,[31],EXP,L'Express,newspaper,Image,"https://dev.impresso-project.ch/media/iiif/EXP-2009-10-17-a-p0031/292,246,242,168/max/0/default.jpg",Scenery or Landscape


#### Use-case B: We want to know similar images to our embedded image

In [14]:
impresso_session.images.find(
  embedding=impresso_session.tools.embed_image(image="https://impresso-project.ch/assets/images/posts/rep-thomas.png", target="image"),
  limit=3
)

,issueUid,previewImage,date,contentItemUid,pageNumbers,mediaSourceRef.uid,mediaSourceRef.name,mediaSourceRef.type,imageTypes.visualContent,imageTypes.visualContentType,previewUrl,caption
uid,,,,,,,,,,,,
IMP-1981-05-08-a-i0016,IMP-1981-05-08-a,,1981-05-08,IMP-1981-05-08-a-i0010,[2],IMP,L'Impartial,newspaper,Image,Human Representation - Scene,"https://dev.impresso-project.ch/media/iiif/IMP-1981-05-08-a-p0002/792,408,1325,907/max/0/default.jpg",nan
EXP-1968-11-21-a-i0451,EXP-1968-11-21-a,,1968-11-21,EXP-1968-11-21-a-i0440,[24],EXP,L'Express,newspaper,Image,Human Representation - Scene,"https://dev.impresso-project.ch/media/iiif/EXP-1968-11-21-a-p0024/631,1394,558,394/max/0/default.jpg",nan
tageblatt-1934-09-24-a-i0064,tageblatt-1934-09-24-a,,1934-09-24,nan,[6],tageblatt,Escher Tageblatt,newspaper,Image,Human Representation - Scene,"https://iiif.eluxemburgensia.lu/image/iiif/2/ark:70795%2f529phf%2fpages%2f6/1021,1589,911,753/max/0/default.jpg","Walter CITRINE, der Generalsekretär der englischen Gewerkschaften. (Photo and Coppright Dr. E. Meyer, London.)"


> For more information on multimodal search and retrieval with Impresso, see the [explore_multimodal-embeddings](https://colab.research.google.com/github/impresso/impresso-datalab-notebooks/blob/main/workshop_resources/ws4-embeddings/explore_multimodal-embeddings.ipynb) notebook!

## Retrieve texts and images embeddings from Impresso


### Retrieve embeddings from a text

Say we found a matching article in Impresso, but **how would we get its embedding**?
Here's a one-liner to make that work.

In [15]:
article_id = "oeuvre-1938-02-25-a-i0003"
original_embedding = impresso_session.content_items.get_embeddings(article_id)[0]
original_embedding

'gte-768:VEeyvW3Jzj1WZfy8cxH8PAJQsL0zPFq8l1QYvVYcaz0onp496uALvRlsFbp61Wc8DOnUPHTpwz2SlB+8gXK/Pe5eUj0iDgQ9WIrIPSy6IL127TY9QxE2PYTlszxalHa8HXvavIv3x7w0LdU9jtphvVVz3b3GSYw99e3IvcEgzTtMahM9jSNzPcdchD19+rO8gP4mPVj6bT2mUSy9TCKmPO5eUjxO/wS9RmPRvFofOruuR/464vvGPPx947z9W+Y8nWfQOjmXf7vsyWA850y+PFZZBjzxg548BSVpPKp7Dz3wiv29Ci3PPDLv1T38v5W81dw1PBlsFbt6EgM7sWzKvZXFYTyf/MG6z4GQuxGjEr3YhJ89hcM2PfoljTwHk8a74aiHO4AJeTqaXSI8QNO2PfhuHjz9NFK8tN++PI0j8zy0Jyy9+ujxPOBhvjzHMuE8UOjZO1g8oDxHaOg8DSuHvKNNuTwcndc7IlysOxhzdDyJXT+7al4AvTkcCD3oA607a//nvHGXKLxpava8LzNQPe+gBD1/K/Y8jSNzPMjCuzt0LBo8du22PBq6vbx4GeK7WBUMvSQ/Rr1aVK+9SKoavMHXO70V9Im7AMkfveNqyD0TWoE9or3evDCA1DyRTVY9p1CIuV1QfDwmqAy9CCOhvDlqsL0aCGY8gp7quuH2r72cYjk9P+dSvey+jj2cuIc9zTpHOW8Icr3vY+k8B0UevGEycr3WoYU9QC9APGM2ZTzVMoS9pCu8PLbCWL2P0wK9qOxYu3ApSz3XmDu81w34u+TTjj3BIE09T0yJPE1IFj2ZEB49R4OGvIldP7wlhzO93sYRve2Azzwj+Pw705VsvDQUIj07kCC7/xdsPGlq9jvpKR29tPsAPRbYRz2tf4I8uzpkvIKearzVjo069iK+PAM8FD2ryJO84GG+vBOGLDzU1x49uYP1PIULJL1UOdG8/ZgBvZHja72TA6G9vPZpu25yXD24ft6

### Retrieve embeddings from an image

We can also **get embeddings for any images in Impresso**.
Be ready! They also come in the quirky encoded format.

In [16]:
image_id = "luxwort-1930-09-26-a-i0036"
embeddings = impresso_session.images.get_embeddings(image_id)
embeddings[0]

'openclip-768:EEyfu8hgLj2pVjA8KxTeu5WGILwSyQ29pliHvbJICD2zTZO8aBpWOpW1qjx1IBm8luGMPFq3SLye9Yw8stBIvX0XiTtK4bk8NHqZvDE7QbuQQlq89jf2u1uGHz21nEg78K0bPJ3T1jwCHk28+12IO8YWujyW2dC80EblO6Qwhbzh8Ia9S4iBvD8siT1a+6g7cOuAPNcVqzuyMbC8fkj/PMwClDzLelM9xwY5vQFd2rtgv6C8xEZIPJso8LuuzLi9DgQ/vOFwHbz04cW7iJTXvLSNkzxNEJE9fIfWvKsLpLkgw5m82iKCvLo8RzypgIA80T3LvCK6iLyCihg9QvgSPMacVDzhf4E7HP3BuvbuZj0gcnA8hZ7fvDMl+by19Ac9DbA7OxVSmzwv2OK7c0HwuyoD6Dvsi528XaQUPH9HGDyxb0Q9canNPJA3kzzoOuq8Hkg2vcfo0zxv3Aq9RkRfvf6MDr2br907fIHtPNXBRD3EYbS8b2nfPEc92T2ni4C9O8EMvCGp2rqjqJc84OkWvWW23DuBRK88ACIvPSLvwzx73jk6rYo2PHVv7bvt3gU75onkvOvMBj1a7807doH1vAvtWrzH7c476oUvPAn+wryrP2i8elWEPE1NrjzQkS69CK3RvKlHnj1tTko84kpxPC2QhDzAFYM8MvqqvDOs+DpfNs48VCOlOxgndDrjpoO9dgckPIXKaTz2TRi9Z7iAPf7CJbysuwO9a0UHvc7MOT3HnpU8Z2osO3KfRbwk5ru7ClovvN+imbz305S9f0eGO0vmHruBgG489hQMPlntVjwFaBu8gZo2PLj1lbw4g8+8+p94vKy8IDsYx+i6xrccPA8pFL1/5yc8JHzDu1TtXrt7kV69sEZbPSyZAT3f4wA9252dvMWk/737QSW8umVEPOrvZjy2omU9Ze4ivZ3EH70bzgO9tY3kO+G6ijy4KjY86MEEPUfiq7zJDWi8OpNOOyuYeL

### Retrieve an image from embeddings

Tired of looking at encodings and vectors?
Let's learn **how we can directly retrieve the image that's represented by the embedding**!

In [17]:
image_id = "luxwort-1930-09-26-a-i0036"
image = impresso_session.images.get(image_id, include_embeddings=True)
image

## Conclusion

In this notebook, we explored **how Impresso’s embedding ecosystem transforms text and images into meaningful vector representations**, enabling:

- cross-lingual retrieval
- similarity search
- multimodal linking

By experimenting with simple similarity functions and the Impresso API, you have seen how embeddings reveal latent relationships beyond surface wording. These techniques provide a foundation for more advanced workflows, such as **document clustering, cross-media analysis, and building richer historical research pipelines**.

---
## Project and License info

### Notebook credits [CreditLogo.png](https://credit.niso.org/)

**Writing - Original draft:**  Roman Kalyakin. **Conceptualization:** Marten Düring. **Software:** Roman Kalyakin. **Writing - Review & Editing**: Juri Opitz, Cao Vy. **Validation:** Martin Grandjean. **Datalab editorial board:** Caio Mello (Managing), Cao Vy, Pauline Conti, Emanuela Boros, Marten Düring, Juri Opitz, Martin Grandjean, Estelle Bunout. **Data curation & Formal analysis:** Maud Ehrmann, Emanuela Boros, Pauline Conti, Simon Clematide, Juri Opitz, Andrianos Michail. **Methodology:** Roman Kalyakin. **Supervision:** Marten Düring. **Funding aquisition:** Maud Ehrmann, Simon Clematide, Marten Düring, Raphaëlle Ruppen Coutaz.

<br><a target="_blank" href="https://creativecommons.org/licenses/by/4.0/">
  <img src="https://mirrors.creativecommons.org/presskit/buttons/88x31/png/by.png"  width="100" alt="Open In Colab"/>
</a> 

This notebook is published under [CC BY 4.0 License](https://creativecommons.org/licenses/by/4.0/)

For feedback on this notebook, please send an email to info@impresso-project.ch

### Impresso project

[Impresso - Media Monitoring of the Past](https://impresso-project.ch) is an interdisciplinary research project that aims to develop and consolidate tools for processing and exploring large collections of media archives across modalities, time, languages and national borders. The first project (2017-2021) was funded by the Swiss National Science Foundation under grant No. [CRSII5_173719](http://p3.snf.ch/project-173719) and the second project (2023-2027) by the SNSF under grant No. [CRSII5_213585](https://data.snf.ch/grants/grant/213585) and the Luxembourg National Research Fund under grant No. 17498891.
<br></br>
### License

All Impresso code is published open source under the [GNU Affero General Public License](https://github.com/impresso/impresso-pyindexation/blob/master/LICENSE) v3 or later.


---

<p align="center">
  <img src="https://github.com/impresso/impresso.github.io/blob/master/assets/images/3x1--Yellow-Impresso-Black-on-White--transparent.png?raw=true" width="350" alt="Impresso Project Logo"/>
</p>
